In [4]:
import pandas as pd

import os
import re

from pathlib import Path
from typing import *

import sys
sys.path.append('..')

from src import utils

%load_ext autoreload
%autoreload 2 

In [7]:
sheet_name = 'rep_hiv_avrt_analyze'

In [8]:
inp = utils.read_excel('../examples/input.xlsx')

/Users/olegpavlovich/Projects/table-converter/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


inp = inp.fillna(method='ffill', axis=0)
inp.head()

In [75]:
test_cases = []
test_cases.append(["""
    |a| | | | |
    |b| | | | |
    |c| | | | |
""", (0, 0, 0)
])
test_cases.append(["""
    |a| | | | |
    | |b|c|d| |
    |e| | | | |
    |f| |x| | |
""", (1, 1, 3)
])
test_cases.append(["""
    |a| | | | |
    | |b| |d| |
    |e| | | | |
    |f| |x| | |
""", (0, 0, 0)
])
test_cases.append(["""
    |a| | | | |
    | |b| |d| |
    |b|c|d| | |
    |f| |x| |y|
""", (2, 0, 2)
])

test_cases.append(["""
    |a| | | | |
    | |b| |d| |
    |b| |d|e|f|
    |f| |x| |y|
""", (2, 2, 4)
])

In [76]:
import textwrap
import io


def read_md(table: str, na_token=' '):
    table = io.StringIO(textwrap.dedent(table).strip())
    table = (
        pd
        .read_table(table, sep='|', header=None, index_col=None)
        .dropna(axis=1, how='all')
    )
    return table.apply(
        lambda x: x.apply(lambda x: x if x != na_token else pd.NA), 
        axis=1
    )

In [103]:
def evaluate_test_cases(test_cases, func):
    for i, test_case in enumerate(test_cases):
        tab, out = test_case
        res = func(read_md(tab))
        if res != out:
            print('-' * 20)
            print("TEST %d NOT PASSED!" % i)
            print('Input: %s' % textwrap.dedent(tab))
            print('Answer: %s' % str(out))
            print('Output: %s' % str(res))

In [78]:
def header_coordinates(data: pd.DataFrame) -> Tuple:
    return (0, 0, 0)

In [42]:
evaluate_test_cases(test_cases, header_coordinates)

--------------------
TEST 1 NOT PASSED!
Input: 
|a| | | | |
| |b|c|d| |
|e| | | | |
|f| |x| | |

Answer: (1, 1, 3)
Output: (0, 0, 0)
--------------------
TEST 3 NOT PASSED!
Input: 
|a| | | | |
| |b| |d| |
|b|c|d| | |
|f| |x| |y|

Answer: (2, 0, 2)
Output: (0, 0, 0)


In [83]:
def leftmost_span(row) -> Tuple[int, int]:
    n = len(row)
    last_span = ()
    for i, val in enumerate(row):
        if val == 0:
            continue
        j = i
        while j < n and row[j]:
            j += 1
        span = i, j - 1
        if width(span) > width(last_span):
            last_span = span
    return last_span

def total_span(row) -> Tuple[int, int]:
    try:
        i = row.index(1)
    except ValueError:
        return tuple()
    j = len(row) - row[::-1].index(1) - 1
    return i, max(j, i)

def width(span: Tuple[int, int]) -> int:
    if not span:
        return 0
    return span[1] - span[0] + 1

In [98]:
def header_coordinates(data: pd.DataFrame, search_nrows=100):
    """Returns the indices of row and cols that are more likely to be a header 
    in a sparse table with lots of empty cells. 
    
    Header in this case is the longest sequence of non-empty cells.

    Args:
        data: A sparse dataframe with lots of empty cells.
        search_nrows: How many rows to go through with a linear search.
    Returns:
        A tuple with the row, start column, end column indices.
    """
    
    header = tuple()
    for i, row in enumerate(data.itertuples(index=False)):
        # print('--')
        # print(f'row {i}')
        brow = tuple(map(int, map(pd.notna, row)))
        lspan = leftmost_span(brow)
        # tspan = total_span(brow)
        # print(f'lspan: {lspan}, tspan: {tspan}')
        if width(lspan) > width(header[1:]):
            header = (i, *lspan)
            print(f':assign -> {header}')
        else:
            print(f':no header')
        if i == search_nrows - 1:
            break
    return header


In [105]:
evaluate_test_cases(test_cases, utils.find_header)

In [70]:
hcoords = header_coordinates(inp)

row 0: assign -> (0, 1, 1)
row 1: no header
row 2: no header
row 3: no header
row 4: no header
row 5: no header
row 6: no header
row 7: no header
row 8: assign -> (8, 1, 12)
row 9: no header
row 10: no header
row 11: no header
row 12: no header
row 13: no header
row 14: no header
row 15: no header
row 16: no header
row 17: no header
row 18: no header
row 19: no header
row 20: no header
row 21: no header
row 22: no header
row 23: no header
row 24: no header
row 25: no header
row 26: no header
row 27: no header
row 28: no header
row 29: no header
row 30: no header
row 31: no header
row 32: no header
row 33: no header
row 34: no header
row 35: no header
row 36: no header
row 37: no header
row 38: no header
row 39: no header
row 40: no header
row 41: no header
row 42: no header
row 43: no header
row 44: no header
row 45: no header
row 46: no header
row 47: no header
row 48: no header
row 49: no header
row 50: no header
row 51: no header
row 52: no header
row 53: no header
row 54: no header

In [101]:
out = utils.remove_useless_cells(inp)
out

,Субъект РФ,Ведомственная принадлежность,Медицинская организация,УНРЗ,Дата включения в регистр,МНН,Лекарственная форма,Дозировка,ЕИ,Потребность на день (ЕИ),Потребность на год (ЕИ)
0,Томская область,Другое ведомство,"ОГБУЗ ""Томский областной центр по профилактике...",0110317027749,28.03.2017,Атазанавир,капсулы,300 мг,капсула,1.000,365.000
1,NaN,NaN,NaN,NaN,NaN,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
2,NaN,NaN,NaN,NaN,NaN,Ритонавир,таблетки покрытые пленочной оболочкой,100 мг,таблетка,1.000,365.000
3,NaN,NaN,NaN,NaN,NaN,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
4,Томская область,Другое ведомство,"ОГБУЗ ""Томский областной центр по профилактике...",0210417041945,25.04.2017,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
...,...,...,...,...,...,...,...,...,...,...,...
17965,NaN,NaN,NaN,NaN,NaN,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
17966,NaN,NaN,NaN,NaN,NaN,Эфавиренз,таблетки покрытые пленочной оболочкой,600 мг,таблетка,1.000,365.000
17967,Томская область,Другое ведомство,"ОГБУЗ ""Томский областной центр по профилактике...",8610717616765,24.07.2017,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
17968,NaN,NaN,NaN,NaN,NaN,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000


In [67]:
new_inp = inp.iloc[hcoords[0] + 1:, hcoords[1]:hcoords[2]].copy()
new_inp.columns = inp.iloc[hcoords[0], hcoords[1]:hcoords[2]].values

In [68]:
new_inp.reset_index(drop=True)

,Субъект РФ,Ведомственная принадлежность,Медицинская организация,УНРЗ,Дата включения в регистр,МНН,Лекарственная форма,Дозировка,ЕИ,Потребность на день (ЕИ),Потребность на год (ЕИ)
0,Томская область,Другое ведомство,"ОГБУЗ ""Томский областной центр по профилактике...",0110317027749,28.03.2017,Атазанавир,капсулы,300 мг,капсула,1.000,365.000
1,NaN,NaN,NaN,NaN,NaN,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
2,NaN,NaN,NaN,NaN,NaN,Ритонавир,таблетки покрытые пленочной оболочкой,100 мг,таблетка,1.000,365.000
3,NaN,NaN,NaN,NaN,NaN,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
4,Томская область,Другое ведомство,"ОГБУЗ ""Томский областной центр по профилактике...",0210417041945,25.04.2017,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
...,...,...,...,...,...,...,...,...,...,...,...
17965,NaN,NaN,NaN,NaN,NaN,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
17966,NaN,NaN,NaN,NaN,NaN,Эфавиренз,таблетки покрытые пленочной оболочкой,600 мг,таблетка,1.000,365.000
17967,Томская область,Другое ведомство,"ОГБУЗ ""Томский областной центр по профилактике...",8610717616765,24.07.2017,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000
17968,NaN,NaN,NaN,NaN,NaN,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.000,365.000


In [50]:
inp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17970 entries, 0 to 17969
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   УНРЗ                       17970 non-null  object 
 1   ДатаВключенияВРегистр      17970 non-null  object 
 2   МНН                        17970 non-null  object 
 3   ЛекарственнаяФорма         17970 non-null  object 
 4   Дозировка                  17970 non-null  object 
 5   ЕИ                         17970 non-null  object 
 6   ПотребностьНаДеньЕИ        17970 non-null  float64
 7   ПотребностьНаГодЕИ         17970 non-null  float64
 8   ЧислоЧеловекНаДаннойСхеме  17970 non-null  float64
dtypes: float64(3), object(6)
memory usage: 1.2+ MB


In [9]:
out = inp.copy()
out['МНН+Дозировка'] = inp[['МНН', 'Дозировка']].apply(lambda r: ', '.join((r.МНН, r.Дозировка)), axis=1)
out['Схема на УРНЗ'] = out.groupby('УНРЗ')['МНН+Дозировка'].transform(lambda x: '; '.join(x))
out

,УНРЗ,Дата включения в регистр,МНН,Лекарственная форма,Дозировка,ЕИ,Потребность на день (ЕИ),Потребность на год (ЕИ),Число человек на данной схеме,МНН+Дозировка,Схема на УРНЗ
0,0110317027749,28.03.2017,Атазанавир,капсулы,300 мг,капсула,1.0,365.0,272.0,"Атазанавир, 300 мг","Атазанавир, 300 мг; Ламивудин, 300 мг; Ритонав..."
1,0110317027749,28.03.2017,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.0,365.0,272.0,"Ламивудин, 300 мг","Атазанавир, 300 мг; Ламивудин, 300 мг; Ритонав..."
2,0110317027749,28.03.2017,Ритонавир,таблетки покрытые пленочной оболочкой,100 мг,таблетка,1.0,365.0,272.0,"Ритонавир, 100 мг","Атазанавир, 300 мг; Ламивудин, 300 мг; Ритонав..."
3,0110317027749,28.03.2017,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.0,365.0,272.0,"Тенофовир, 300 мг","Атазанавир, 300 мг; Ламивудин, 300 мг; Ритонав..."
4,0210417041945,25.04.2017,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.0,365.0,2079.0,"Ламивудин, 300 мг","Ламивудин, 300 мг; Тенофовир, 300 мг; Эфавирен..."
...,...,...,...,...,...,...,...,...,...,...,...
17965,8610617247498,08.06.2017,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.0,365.0,2079.0,"Тенофовир, 300 мг","Ламивудин, 300 мг; Тенофовир, 300 мг; Эфавирен..."
17966,8610617247498,08.06.2017,Эфавиренз,таблетки покрытые пленочной оболочкой,600 мг,таблетка,1.0,365.0,2079.0,"Эфавиренз, 600 мг","Ламивудин, 300 мг; Тенофовир, 300 мг; Эфавирен..."
17967,8610717616765,24.07.2017,Ламивудин,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.0,365.0,2079.0,"Ламивудин, 300 мг","Ламивудин, 300 мг; Тенофовир, 300 мг; Эфавирен..."
17968,8610717616765,24.07.2017,Тенофовир,таблетки покрытые пленочной оболочкой,300 мг,таблетка,1.0,365.0,2079.0,"Тенофовир, 300 мг","Ламивудин, 300 мг; Тенофовир, 300 мг; Эфавирен..."


In [12]:
# todo:
# 1. сделать вывод результатов файл с тремя страницами
# 2. сделать простенький GUI:
#    - выбор исходного файла и названием целевого листа на нем
#    - окошко со статусом преобразования / какой-то информацией об ошибках
#    - выбор пути для сохранения нового файла

out_path = Path('../outputs/')
out_path.mkdir(parents=True, exist_ok=True)
out_file = out_path / 'out.xlsx'

try:
    os.remove(out_file)
except FileNotFoundError:
    pass

In [13]:
out.to_excel(out_file, sheet_name=sheet_name, index=False)

In [14]:
import win32com.client as win32

In [17]:
# create Excel object
excel = win32.gencache.EnsureDispatch('Excel.Application')
# excel can be visible or not
excel.Visible = True 
# open Excel Workbook   
wb = excel.Workbooks.Open(out_file.resolve())

In [19]:
wb.__class__

win32com.gen_py.00020813-0000-0000-C000-000000000046x0x1x9.Workbook.Workbook

In [25]:
win32c = win32.constants

In [26]:
type(win32c)

win32com.client.Constants

In [5]:
class PTField(NamedTuple):
    field: str
    name: str
    calculation: object
    number_format: str = '0'

In [6]:
class PTFields(NamedTuple):
    values: List[PTField]
    rows: List[str] = tuple()
    columns: List[str] = tuple()
    filters: List[str] = tuple()

In [21]:
def create_new_sheet(wb: object, name: str) -> object:
    wb.Sheets.Add().Name = name
    return wb.Sheets(name)

In [79]:
def pivot_table(wb: object, ws: object, pt_ws: object, fields: PTFields, show_annotations: bool = False):
    """
    wb = workbook1 reference
    ws = worksheet1
    pt_ws = pivot table worksheet
    ws_name = pivot table worksheet name
    pt_name = name given to pivot table
    pt_rows, pt_cols, pt_filters, pt_fields: values selected for filling the pivot tables
    """

    # pivot table location
    pt_loc = len(fields.filters) + 2
    
    # grab the pivot table source data
    pc = wb.PivotCaches().Create(SourceType=win32c.xlDatabase, SourceData=ws.UsedRange)
    
    # create the pivot table object
    pc.CreatePivotTable(TableDestination=f"'{pt_ws.Name}'!R{pt_loc}C1", TableName=pt_ws.Name)

    # selecte the pivot table work sheet and location to create the pivot table
    pt_ws.Select()
    pt_ws.Cells(pt_loc, 1).Select()

    # Sets the rows, columns and filters of the pivot table
    for field_list, field_r in ((fields.filters, win32c.xlPageField), (fields.rows, win32c.xlRowField), (fields.columns, win32c.xlColumnField)):
        for i, value in enumerate(field_list):
            pt_ws.PivotTables(pt_ws.Name).PivotFields(value).Orientation = field_r
            pt_ws.PivotTables(pt_ws.Name).PivotFields(value).Position = i + 1

    # Sets the Values of the pivot table
    for value in fields.values:
        field = (
            pt_ws.PivotTables(pt_ws.Name)
            .AddDataField(
                pt_ws.PivotTables(pt_ws.Name).PivotFields(value.field), 
                value.name, 
                value.calculation
            )
        )
        field.NumberFormat = value.number_format

    # Visiblity True or Valse
    pt_ws.PivotTables(pt_ws.Name).ShowValuesRow = show_annotations
    pt_ws.PivotTables(pt_ws.Name).ColumnGrand = show_annotations

In [8]:
fields = PTFields(
    rows=['МНН+Дозировка'],
    values=[
        PTField('УНРЗ', 'Количество по полю УНРЗ', 'count'),
        PTField('Потребность на год (ЕИ)', 'Сумма по полю Потребность на год (ЕИ)', 'sum'),
    ],
)

In [15]:
table = type('dummy', (object,), {})()
table.fields = fields

In [16]:
utils.get_field_dependencies(table)

{'МНН+Дозировка', 'Потребность на год (ЕИ)', 'УНРЗ'}

In [83]:
ws = wb.Sheets(sheet_name)

In [84]:
pt_ws = create_new_sheet(wb, 'Потребность в препаратах')

In [85]:
pivot_table(wb, ws, pt_ws, fields)

# Done

In [67]:
out_1 = inp.copy()
out_1['МНН+Дозировка'] = inp[['МНН', 'Дозировка']].apply(lambda r: ', '.join((r.МНН, r.Дозировка)), axis=1)

In [11]:
inverse_columns_map = {v: k for k, v in columns_map.items()}
agg_columns_map = {
    'Количество по полю УНРЗ': ('УНРЗ', 'count'),
    'Сумма по полю Потребность на год (ЕИ)': ('ПотребностьНаГодЕИ', 'sum'),
}

out_1 = (out_1
    .groupby('МНН+Дозировка')
    .agg(**agg_columns_map)
    .reset_index()
)

In [48]:
out_2 = inp.copy()
out_2['МНН+Дозировка'] = inp[['МНН', 'Дозировка']].apply(lambda r: ', '.join((r.МНН, r.Дозировка)), axis=1)

In [49]:
out_2 = (
    out_2
    .groupby('УНРЗ')
    .agg({'МНН+Дозировка': lambda v: '; '.join(v)})
    .reset_index()
    .groupby('МНН+Дозировка')
    .agg({'УНРЗ': ('count', ('Список', lambda v: ', '.join(v)))})
    .reset_index()
    .rename(columns={'count': 'Количество'})
    .reset_index(drop=True)
)
out_2

МНН+Дозировка       УНРЗ  \
                                                       Количество   
0    Абакавир + Ламивудин, 600 мг + 300 мг; Атазана...          7   
1    Абакавир + Ламивудин, 600 мг + 300 мг; Атазана...          4   
2    Абакавир + Ламивудин, 600 мг + 300 мг; Дарунав...          5   
3    Абакавир + Ламивудин, 600 мг + 300 мг; Долутег...          3   
4    Абакавир + Ламивудин, 600 мг + 300 мг; Лопинав...         21   
..                                                 ...        ...   
116  Ламивудин, 300 мг; Фосфазид, 400 мг; Эфавиренз...          1   
117  Лопинавир + Ритонавир, 200 мг + 50 мг; Ралтегр...          2   
118  Лопинавир + Ритонавир, 200 мг + 50 мг; Тенофов...          1   
119  Рилпивирин+ Тенофовир+ Эмтрицитабин, 25 мг + 3...        193   
120  Тенофовир, 300 мг; Эмтрицитабин, 200 мг; Эфави...         13   

                                                        
                                                Список  
0    7010118818801, 7010217005713, 7010313736435, 7...  
1    7010217009003, 7010217012773, 7010217012779, 7...  
2    6610617385511, 7010617253064, 7020617618031, 7...  
3          7010517101928, 7010618878384, 7010816607007  
4    7010118818799, 7010211637908, 7010217005038, 7...  
..                                                 ...  
116                                      7010612576990  
117                     7010117625856, 701022001049565  
118                                      7010515426736  
119  2220513757831, 7010113728932, 7010117626077, 7...  
120  7010118817846, 7010119936308, 701022001051866,...  

[121 rows x 3 columns]

In [42]:
out_2.set_index('МНН+Дозировка')

УНРЗ  \
                                                   Количество   
МНН+Дозировка                                                   
Абакавир + Ламивудин, 600 мг + 300 мг; Атазанав...          7   
Абакавир + Ламивудин, 600 мг + 300 мг; Атазанав...          4   
Абакавир + Ламивудин, 600 мг + 300 мг; Дарунави...          5   
Абакавир + Ламивудин, 600 мг + 300 мг; Долутегр...          3   
Абакавир + Ламивудин, 600 мг + 300 мг; Лопинави...         21   
...                                                       ...   
Ламивудин, 300 мг; Фосфазид, 400 мг; Эфавиренз,...          1   
Лопинавир + Ритонавир, 200 мг + 50 мг; Ралтегра...          2   
Лопинавир + Ритонавир, 200 мг + 50 мг; Тенофови...          1   
Рилпивирин+ Тенофовир+ Эмтрицитабин, 25 мг + 30...        193   
Тенофовир, 300 мг; Эмтрицитабин, 200 мг; Эфавир...         13   

                                                                                                       
                                                                                          Список УНРЗ  
МНН+Дозировка                                                                                          
Абакавир + Ламивудин, 600 мг + 300 мг; Атазанав...  7010118818801, 7010217005713, 7010313736435, 7...  
Абакавир + Ламивудин, 600 мг + 300 мг; Атазанав...  7010217009003, 7010217012773, 7010217012779, 7...  
Абакавир + Ламивудин, 600 мг + 300 мг; Дарунави...  6610617385511, 7010617253064, 7020617618031, 7...  
Абакавир + Ламивудин, 600 мг + 300 мг; Долутегр...        7010517101928, 7010618878384, 7010816607007  
Абакавир + Ламивудин, 600 мг + 300 мг; Лопинави...  7010118818799, 7010211637908, 7010217005038, 7...  
...                                                                                               ...  
Ламивудин, 300 мг; Фосфазид, 400 мг; Эфавиренз,...                                      7010612576990  
Лопинавир + Ритонавир, 200 мг + 50 мг; Ралтегра...                     7010117625856, 701022001049565  
Лопинавир + Ритонавир, 200 мг + 50 мг; Тенофови...                                      7010515426736  
Рилпивирин+ Тенофовир+ Эмтрицитабин, 25 мг + 30...  2220513757831, 7010113728932, 7010117626077, 7...  
Тенофовир, 300 мг; Эмтрицитабин, 200 мг; Эфавир...  7010118817846, 7010119936308, 701022001051866,...  

[121 rows x 2 columns]

In [33]:
# todo:
# 1. сделать вывод результатов файл с тремя страницами
# 2. сделать простенький GUI:
#    - выбор исходного файла и названием целевого листа на нем
#    - окошко со статусом преобразования / какой-то информацией об ошибках
#    - выбор пути для сохранения нового файла

out_path = Path('../outputs/')
out_path.mkdir(parents=True, exist_ok=True)
out_file = out_path / 'out.xlsx'

try:
    os.remove(out_file)
except FileNotFoundError:
    pass

In [28]:
out_1.to_excel(
    out_file,
    sheet_name='Потребность в препаратах',
    # engine=engine,
)
out_2.to_excel(
    out_file,
    sheet_name='Схемы',
    # engine=engine,
)
inp.to_excel(
    out_file,
    sheet_name=sheet_name,
    # engine=engine,
)

In [ ]:
inp

In [36]:
from pandas import ExcelWriter
from typing import Dict


def save_xls(sheets: Dict[str, pd.DataFrame], path: str, **kwargs):
    """Save a dictionary of dataframes to an excel file, with each dataframe as a separate page."""

    with ExcelWriter(path) as writer:
        for sheet, data in sheets.items():
            data.to_excel(excel_writer=writer, sheet_name=sheet, **kwargs)

In [37]:
save_xls(
    sheets={
        'Потребность в препаратах': out_1,
        'Схемы': out_2,
        sheet_name: inp.rename(columns={
            v: k for k, v in columns_map.items()
        })
    },
    path=out_file,
    index=False
)

NotImplementedError: Writing to Excel with MultiIndex columns and no index ('index'=False) is not yet implemented.